In [ ]:
import urllib.request
from zipfile import ZipFile
from lxml import etree
import os
import re
import pandas as pd
from bertopic import BERTopic


def downxml():
    if os.path.isfile('/Users/neele/Downloads/uscode.zip') == 1:
        print("File Already Present!")
    else:
        url = 'https://uscode.house.gov/download/releasepoints/us/pl/117/327not263not286/xml_usc26@117-327not263not286.zip'
        urllib.request.urlretrieve(url, '/Users/neele/Downloads/uscode.zip')

    zf = ZipFile('/Users/neele/Downloads/uscode.zip', 'r')
    zf.extractall('/Users/neele/Downloads/')
    zf.close()
    print('File Extract Success!')

downxml()

tree = etree.parse('/Users/neele/Downloads/usc26.xml')
root = tree.getroot()

root.remove(root[0])
print(root[0].tag)

In [ ]:
ns_dict = {'ns': root.nsmap[None]}
notes = root.xpath("//ns:notes",namespaces = ns_dict)
toc = root.xpath("//ns:toc",namespaces = ns_dict)
note = root.xpath("//ns:note",namespaces = ns_dict)
srcrd = root.xpath("//ns:sourceCredit",namespaces = ns_dict)

In [ ]:
[node.getparent().remove(node) for node in note]
[node.getparent().remove(node) for node in notes]
[node.getparent().remove(node) for node in toc]
[node.getparent().remove(node) for node in srcrd]

In [ ]:
nodes = root.findall(".//ns:section",namespaces = ns_dict)

In [ ]:
len([node.getparent() for node in nodes])

In [ ]:
secs=''
for node in nodes:
    text = ' '.join(node.itertext()).strip()
    text=text+'//////'
    secs = secs+text

sep_pattern = r"\s*//////\s*"


with open('secs_regex1.txt', 'w',encoding='utf-8') as f:

    for line in secs:
        f.write(line)

In [ ]:
with open('secs_regex1.txt', 'r',encoding='utf-8') as f:
    fintxt = f.read()

rows = fintxt.split("//////")

In [ ]:
rows[0]

In [ ]:
def get_snum(row):
    pattern = r'\d.*?(?=\.)'
    match = re.search(pattern, row)
    if match:
       return (match.group())

In [ ]:
def get_text(x):
    dot_index = x.find(".")
    result = x[dot_index + 1:]
    return result

data = {'Title':['26' for row in rows],'Section': [get_snum(row) for row in rows], 'Content': [get_text(row) for row in rows]}

df = pd.DataFrame(data)

df.to_csv('finsec.csv', index=False)

In [ ]:
df.shape

In [ ]:
df

In [ ]:
rdf=df.copy()
repealed_rows = rdf[rdf['Content'].str.contains('Repealed\.  Pub\.') & (rdf['Content'].str.len() <= 100)].index.tolist()

print(repealed_rows)

In [ ]:
rdf = rdf.drop(repealed_rows, axis=0)

In [ ]:
rdf.shape

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

stop_words = stopwords.words('english')

def clean_text(x):
  x = str(x)
  x = x.lower()
  x = re.sub(r'#[A-Za-z0-9]*', ' ', x)
  x = re.sub(r'https*://.*', ' ', x)
  x = re.sub(r'@[A-Za-z0-9]+', ' ', x)
  tokens = word_tokenize(x)
  x = ' '.join([w for w in tokens if not w.lower() in stop_words])
  x = re.sub(r'[%s]' % re.escape('!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~“…”’'), ' ', x)
  x = re.sub(r'\d+', ' ', x)
  x = re.sub(r'\n+', ' ', x)
  x = re.sub(r'\s{2,}', ' ', x)
  return x


rdf['clean_text'] = rdf.Content.apply(clean_text)

rdf.head()

In [ ]:
rdf.reset_index(drop=False, inplace=True)
rdf.rename(columns={'index': 'indno'}, inplace=True)
ndf=rdf.copy()

In [ ]:
ndf.shape

In [ ]:
ndf.columns

In [ ]:
ndf.head(1000)

In [ ]:
ndf.clean_text[4]

In [ ]:
user_input = input("")
user_input2= input('')
result = ndf.loc[ndf['indno'] == int(user_input)]
result1 = ndf.loc[ndf['indno'] == int(user_input2)]
print(result)
print(result1)

In [ ]:
tweets = rdf.clean_text.to_list()

In [ ]:
topic_model = BERTopic(language="english")
topics, probs = topic_model.fit_transform(tweets)

In [ ]:
topic_model._extract_embeddings(tweets)

In [ ]:
topic_model.visualize_documents(tweets)

In [ ]:
topic_model.get_topic_info()

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_barchart()

In [ ]:
topic_model.visualize_heatmap()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
doc_embds = topic_model._extract_embeddings(tweets)

sim_array = cosine_similarity(doc_embds,doc_embds)
similarity_matrix = cosine_similarity(doc_embds)

In [ ]:
len(tweets)

In [ ]:
sim_array

In [ ]:
doc_embds

In [ ]:
lower_indices = []
for x in range(1900):
    for y in range(1900):
        if sim_array[x,y] < 0.1:
            lower_indices.append((x,y))


In [ ]:
len(lower_indices)

In [ ]:
lower_indices

In [ ]:
similarity_matrix

In [ ]:
import pandas as pd
df = pd.DataFrame(np.add(np.where(similarity_matrix < 0.1), 0))

In [ ]:
df = df.T

In [ ]:
df.shape

In [ ]:
df['score'] = sim_array[df[0], df[1]]

In [ ]:
df

In [ ]:
similarity_matrix

In [ ]:
import plotly.express as px
sim_array = cosine_similarity(doc_embds, doc_embds)
px.imshow(sim_array)

In [ ]:
import plotly.express as px
px.imshow(similarity_matrix)

In [ ]:
f = np.load("doc_sim_matrix.npy")

In [ ]:
np.fill_diagonal(similarity_matrix, 0)

similarity_matrix = np.triu(similarity_matrix)

most_similar = np.dstack(np.unravel_index(np.argsort(similarity_matrix.ravel())[::-1], similarity_matrix.shape))[0]
least_similar = np.dstack(np.unravel_index(np.argsort(similarity_matrix.ravel()), similarity_matrix.shape))[0]


doc_ids = ['doc{}'.format(i) for i in range(1900)]
ndf = pd.DataFrame(similarity_matrix, index=doc_ids, columns=doc_ids)

# Print most similar and least similar documents
print('Most similar documents:')
for i, j in most_similar[:50]:
    print('{} - {} : {}'.format(doc_ids[i], doc_ids[j], similarity_matrix[i,j]))
print()

print('Least similar documents:')
for i, j in least_similar[:50]:
    print('{} - {} : {}'.format(doc_ids[i], doc_ids[j], similarity_matrix[i,j]))
print()